In [ ]:
#dependencies
!pip install ultralytics torch torchvision torch-geometric opencv-python tqdm google-generativeai sentence-transformers


In [ ]:
#dataset download
import os
import shutil
import glob
from pathlib import Path
import kagglehub


CLASSES = ["fire", "accident", "normal"]

BASE = Path("data/gnn_images")
BASE.mkdir(parents=True, exist_ok=True)

for c in CLASSES:
    (BASE / c).mkdir(exist_ok=True)

def move_images(src_glob, dst, limit=None):
    files = glob.glob(src_glob, recursive=True)
    if limit:
        files = files[:limit]

    count = 0
    for f in files:
        ext = os.path.splitext(f)[-1].lower()
        if ext not in [".jpg", ".jpeg", ".png"]:
            continue

        new_name = f"{dst}_{count}{ext}"
        shutil.copy(f, BASE / dst / new_name)
        count += 1

    print(f"✔ {count} images added → {dst}")


print("\n⬇ Downloading FIRE dataset...")
fire_path = kagglehub.dataset_download("phylake1337/fire-dataset")

move_images(f"{fire_path}/fire_dataset/fire_images/*.*", "fire")

# Non fire = normal class
move_images(f"{fire_path}/fire_dataset/non_fire_images/*.*", "normal")



print("\n⬇ Downloading ACCIDENT dataset...")
acc_path = kagglehub.dataset_download("ckay16/accident-detection-from-cctv-footage")

# Accident images
move_images(f"{acc_path}/data/train/Accident/*.*", "accident")
move_images(f"{acc_path}/data/val/Accident/*.*", "accident")
move_images(f"{acc_path}/data/test/Accident/*.*", "accident")

# Non-Accident → Normal class
move_images(f"{acc_path}/data/train/Non Accident/*.*", "normal")
move_images(f"{acc_path}/data/val/Non Accident/*.*", "normal")
move_images(f"{acc_path}/data/test/Non Accident/*.*", "normal")


print("\n⬇ Downloading Natural Images dataset...")
nat_path = kagglehub.dataset_download("prasunroy/natural-images")

move_images(f"{nat_path}/**/*.*", "normal", limit=800)

print("\nDataset ready under data/gnn_images/")


In [2]:

# Hybrid GNN Training Script

import os
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from torchvision import models, transforms
from torch_geometric.data import Data
from torch_geometric.nn import GATConv

# Silence ultralytics logs
try:
    from ultralytics.utils import LOGGER as UL_LOGGER
    UL_LOGGER.setLevel(40)
except:
    pass
from ultralytics import YOLO


DATA_DIR = "data/gnn_images"
CLASSES = ["normal", "fire", "accident"]
CLASS_MAP = {c: i for i, c in enumerate(CLASSES)}
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
YOLO_WEIGHTS = "yolov8n.pt"

EPOCHS = 10
LR = 1e-4
BATCH_SIZE = 1
CHECKPOINT_PATH = "hybrid_gnn_checkpoint.pt"

GLOBAL_DIM = 128
LOCAL_DIM = 5
IN_DIM = GLOBAL_DIM + LOCAL_DIM  # 133

print("[CONFIG] DEVICE:", DEVICE)


# Dataset

class GNNImageDataset(Dataset):
    def __init__(self, root):
        self.samples = []
        for cname in CLASSES:
            folder = os.path.join(root, cname)
            imgs = glob.glob(folder + "/*")
            for f in imgs:
                if f.lower().endswith((".jpg", ".jpeg", ".png")):
                    self.samples.append((f, CLASS_MAP[cname]))

        print(f"[DATASET] Loaded {len(self.samples)} images.")

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        img_np = np.array(img).astype("uint8")

        if img_np.ndim == 4:
            img_np = img_np.squeeze()
        if img_np.ndim == 2:
            img_np = np.stack([img_np]*3, axis=-1)
        if img_np.shape[-1] != 3:
            img_np = img_np[..., :3]

        return img_np, label, path

    def __len__(self):
        return len(self.samples)


# Load ResNet Backbone
resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet.fc = nn.Identity()
resnet = resnet.to(DEVICE)
resnet.eval()
for p in resnet.parameters():
    p.requires_grad = False

compress = nn.Linear(512, GLOBAL_DIM).to(DEVICE)

img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])


# YOLO inference
def yolo_infer(model, img_np):
    pil = Image.fromarray(img_np)
    with torch.no_grad():
        out = model.predict(pil, verbose=False, device=DEVICE)
    return out[0]


# Build Graph
def build_graph(image_np, yres):
    H, W = image_np.shape[:2]

    # global embedding
    with torch.no_grad():
        t = img_transform(Image.fromarray(image_np)).unsqueeze(0).to(DEVICE)
        feat = resnet(t)
    global_vec = compress(feat).squeeze(0)

    boxes = getattr(yres, "boxes", None)

    if boxes is None or len(boxes) == 0:
        local = torch.zeros((LOCAL_DIM,), device=DEVICE)
        x = torch.cat([global_vec, local]).unsqueeze(0)
        edge_index = torch.tensor([[0],[0]], dtype=torch.long, device=DEVICE)
        return Data(x=x.float(), edge_index=edge_index)

    xyxy = boxes.xyxy.cpu().numpy()
    conf = boxes.conf.cpu().numpy()
    cls_ids = boxes.cls.cpu().numpy()

    nodes = []
    for i in range(len(xyxy)):
        x1,y1,x2,y2 = xyxy[i]
        cx = (x1 + x2) / 2 / W
        cy = (y1 + y2) / 2 / H
        area = ((x2 - x1)*(y2 - y1)) / (W*H)

        local_feat = torch.tensor([cls_ids[i], conf[i], cx, cy, area], device=DEVICE)
        full_vec = torch.cat([global_vec, local_feat])
        nodes.append(full_vec)

    x = torch.stack(nodes)
    N = x.size(0)
    if N == 1:
        edge_index = torch.tensor([[0],[0]], dtype=torch.long, device=DEVICE)
    else:
        edges = [[i,j] for i in range(N) for j in range(N) if i != j]
        edge_index = torch.tensor(edges, dtype=torch.long, device=DEVICE).t()

    return Data(x=x.float(), edge_index=edge_index)


# Model
class HybridGAT(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(IN_DIM, 256, heads=2)       # -> 512
        self.conv2 = GATConv(512, 256, heads=1)          # -> 256
        self.fc = nn.Linear(256, len(CLASSES))

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = x.mean(dim=0, keepdim=True)
        return self.fc(x)


# Initialize
yolo = YOLO(YOLO_WEIGHTS)
yolo.to(DEVICE)

dataset = GNNImageDataset(DATA_DIR)

labels = [lbl for _, lbl, _ in dataset]
counts = Counter(labels)
weights = torch.DoubleTensor([1.0/np.sqrt(counts[lbl]) for lbl in labels])

sampler = WeightedRandomSampler(weights, len(weights), replacement=True)
loader = DataLoader(dataset, batch_size=1, sampler=sampler, num_workers=2)

model = HybridGAT().to(DEVICE)
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(compress.parameters()),
    lr=LR
)
criterion = nn.CrossEntropyLoss()

best_acc = 0.0

print("\n===== START TRAINING =====\n")

for epoch in range(1, EPOCHS + 1):
    total_loss = 0
    correct = 0
    total = 0

    pbar = tqdm(loader, desc=f"Epoch {epoch}/{EPOCHS}")
    for img_np, label, path in pbar:
        img_np = img_np[0].numpy()

        yres = yolo_infer(yolo, img_np)

        graph = build_graph(img_np, yres).to(DEVICE)

        lbl = torch.tensor([label], device=DEVICE)

        optimizer.zero_grad()
        logits = model(graph.x, graph.edge_index)
        loss = criterion(logits, lbl)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = logits.argmax(1).item()
        correct += (pred == label)
        total += 1

        pbar.set_postfix({
            "loss": float(total_loss / total),
            "acc": float(correct / total)
        })


    acc = correct / total
    print(f"[EPOCH {epoch}] Loss={float(total_loss/total):.4f}  Acc={float(acc):.4f}")

    if float(acc) > best_acc:
        best_acc = float(acc)
        torch.save({
            "model_state": model.state_dict(),
            "compress_state": compress.state_dict(),
            "epoch": epoch,
            "best_acc": best_acc,
            "classes": CLASSES
        }, CHECKPOINT_PATH)
        print(f"[INFO] Saved best checkpoint ({best_acc:.4f})")


print("\n===== TRAINING COMPLETE =====")
print("Best accuracy =", best_acc)


[CONFIG] DEVICE: cuda
[DATASET] Loaded 2168 images.



===== START TRAINING =====



Epoch 1/10: 100%|██████████| 2168/2168 [01:12<00:00, 29.72it/s, loss=0.34, acc=0.833] 


[EPOCH 1] Loss=0.3396  Acc=0.8335
[INFO] Saved best checkpoint (0.8335)


Epoch 2/10: 100%|██████████| 2168/2168 [01:09<00:00, 31.13it/s, loss=0.191, acc=0.922]


...

[EPOCH 8] Loss=0.1053  Acc=0.9599
[INFO] Saved best checkpoint (0.9599)


Epoch 9/10: 100%|██████████| 2168/2168 [01:09<00:00, 31.03it/s, loss=0.128, acc=0.955]


[EPOCH 9] Loss=0.1278  Acc=0.9553


Epoch 10/10: 100%|██████████| 2168/2168 [01:10<00:00, 30.81it/s, loss=0.0877, acc=0.962]


[EPOCH 10] Loss=0.0877  Acc=0.9622
[INFO] Saved best checkpoint (0.9622)

===== TRAINING COMPLETE =====
Best accuracy = 0.9621770977973938


In [3]:
# Hybrid GNN Evaluation

import os
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models, transforms
from torch_geometric.data import Data
from torch_geometric.nn import GATConv

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

from ultralytics import YOLO


DATA_DIR = "data/gnn_images"
CHECKPOINT_PATH = "hybrid_gnn_checkpoint.pt"
CLASSES = ["normal", "fire", "accident"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GLOBAL_DIM = 128
LOCAL_DIM = 5
IN_DIM = GLOBAL_DIM + LOCAL_DIM


# Dataset
class EvalDataset(torch.utils.data.Dataset):
    def __init__(self, root):
        self.samples = []
        for cname in CLASSES:
            folder = os.path.join(root, cname)
            imgs = glob.glob(folder + "/*")
            for f in imgs:
                if f.lower().endswith((".jpg", ".jpeg", ".png")):
                    self.samples.append((f, CLASSES.index(cname)))

        print(f"[EVAL] Loaded {len(self.samples)} images.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        img_np = np.array(img).astype("uint8")
        return img_np, label, path


# Backbones
resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet.fc = nn.Identity()
resnet.to(DEVICE)
resnet.eval()
for p in resnet.parameters(): p.requires_grad = False

compress = nn.Linear(512, GLOBAL_DIM).to(DEVICE)
compress.eval()

img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])


# Silent YOLO
yolo = YOLO("yolov8n.pt")
yolo.to(DEVICE)

def yolo_infer(img_np):
    pil = Image.fromarray(img_np)
    with torch.no_grad():
        return yolo.predict(pil, verbose=False, device=DEVICE)[0]


# Graph builder (same as training)
def build_graph(image_np, yres):
    H, W = image_np.shape[:2]
    with torch.no_grad():
        t = img_transform(Image.fromarray(image_np)).unsqueeze(0).to(DEVICE)
        feat = resnet(t)
    global_vec = compress(feat).squeeze(0)

    boxes = getattr(yres, "boxes", None)
    if boxes is None or len(boxes) == 0:
        local = torch.zeros((LOCAL_DIM,), device=DEVICE)
        x = torch.cat([global_vec, local]).unsqueeze(0)
        edge_index = torch.tensor([[0],[0]], dtype=torch.long, device=DEVICE)
        return Data(x=x.float(), edge_index=edge_index)

    xyxy = boxes.xyxy.cpu().numpy()
    conf = boxes.conf.cpu().numpy()
    cls_ids = boxes.cls.cpu().numpy()

    nodes = []
    for i in range(len(xyxy)):
        x1,y1,x2,y2 = xyxy[i]
        cx = (x1+x2)/2/W
        cy = (y1+y2)/2/H
        area = ((x2-x1)*(y2-y1))/(W*H)

        local = torch.tensor([cls_ids[i], conf[i], cx, cy, area], device=DEVICE)
        node = torch.cat([global_vec, local])
        nodes.append(node)

    x = torch.stack(nodes).float()

    N = x.size(0)
    if N == 1:
        edge_index = torch.tensor([[0],[0]], device=DEVICE)
    else:
        edges = [[i,j] for i in range(N) for j in range(N) if i != j]
        edge_index = torch.tensor(edges, dtype=torch.long, device=DEVICE).t()

    return Data(x=x, edge_index=edge_index)


# GNN Model
class HybridGAT(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(IN_DIM, 256, heads=2)
        self.conv2 = GATConv(512, 256, heads=1)
        self.fc = nn.Linear(256, len(CLASSES))

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = x.mean(dim=0, keepdim=True)
        return self.fc(x)


# Load checkpoint
ckpt = torch.load(CHECKPOINT_PATH, map_location=DEVICE)

model = HybridGAT().to(DEVICE)
model.load_state_dict(ckpt["model_state"])

compress.load_state_dict(ckpt["compress_state"])

model.eval()
compress.eval()


# Evaluation loop
dataset = EvalDataset(DATA_DIR)
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

y_true = []
y_pred = []

print("\n[EVAL] Running evaluation...\n")

for img_np, label, path in tqdm(loader):
    img_np = img_np[0].numpy()
    label = int(label)

    yres = yolo_infer(img_np)
    graph = build_graph(img_np, yres).to(DEVICE)

    with torch.no_grad():
        logits = model(graph.x, graph.edge_index)
        pred = logits.argmax().item()

    y_true.append(label)
    y_pred.append(pred)


# Metrics
acc = accuracy_score(y_true, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="macro", zero_division=0)
cm = confusion_matrix(y_true, y_pred)

print("\n===== RESULTS =====")
print("Accuracy :", acc)
print("Precision:", prec)
print("Recall   :", rec)
print("F1 Score :", f1)

print("\nConfusion Matrix:")
print(cm)


[EVAL] Loaded 2168 images.

[EVAL] Running evaluation...



100%|██████████| 2168/2168 [01:14<00:00, 29.11it/s]



===== RESULTS =====
Accuracy : 0.9746309963099631
Precision: 0.9626976724178463
Recall   : 0.9719262997227673
F1 Score : 0.9671089083058789

Confusion Matrix:
[[1007    1   36]
 [   0  755    0]
 [  18    0  351]]


In [4]:
import kagglehub
import shutil
import os
import glob

# Download the dataset
path = kagglehub.dataset_download("smrutisanchitadas/flame-dataset-fire-classification")
print("Dataset downloaded at:", path)

test_root = os.path.join(path, "Training", "Training")

src_fire = os.path.join(test_root, "Fire")
src_non_fire = os.path.join(test_root, "No_Fire")

dst_root = "data/eval_fire/train"
dst_fire = os.path.join(dst_root, "fire")
dst_non_fire = os.path.join(dst_root, "non_fire")

os.makedirs(dst_fire, exist_ok=True)
os.makedirs(dst_non_fire, exist_ok=True)

print("Copying Fire images...")
for f in glob.glob(src_fire + "/*"):
    if f.lower().endswith((".jpg", ".jpeg", ".png")):
        shutil.copy(f, dst_fire)

print("Copying No_Fire images...")
for f in glob.glob(src_non_fire + "/*"):
    if f.lower().endswith((".jpg", ".jpeg", ".png")):
        shutil.copy(f, dst_non_fire)

print("DONE. Evaluation dataset created at:", dst_root)


Dataset downloaded at: /teamspace/studios/this_studio/.cache/kagglehub/datasets/smrutisanchitadas/flame-dataset-fire-classification/versions/2
Copying Fire images...
Copying No_Fire images...
DONE. Evaluation dataset created at: data/eval_fire/train


In [1]:
# ============================================================
#   FIXED HYBRID GNN EVALUATION FOR FIRE/NON_FIRE DATASET
#   (Checkpoint trained on 3 classes → normal, fire, accident)
# ============================================================

import os
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from torch_geometric.data import Data
from torch_geometric.nn import GATConv

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

from ultralytics import YOLO


# ---------------- CONFIG ----------------
TEST_DIR = "data/eval_fire"  # fire / non_fire
CHECKPOINT_PATH = "hybrid_gnn_checkpoint.pt"

TRAIN_CLASSES = ["normal", "fire", "accident"]   # original model classes
EVAL_CLASSES  = ["non_fire", "fire"]             # evaluation classes

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
GLOBAL_DIM = 128
LOCAL_DIM = 5
IN_DIM = GLOBAL_DIM + LOCAL_DIM

print("DEVICE =", DEVICE)


# -------------- Dataset ------------------
class EvalDataset(torch.utils.data.Dataset):
    def __init__(self, root):
        self.samples = []
        for cname, label in [("fire",1), ("non_fire",0)]:
            folder = os.path.join(root, cname)
            imgs = glob.glob(folder + "/*")
            for f in imgs:
                if f.lower().endswith((".jpg",".png",".jpeg")):
                    self.samples.append((f, label))

        print(f"[EVAL] Loaded {len(self.samples)} samples.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        img_np = np.array(img).astype("uint8")
        return img_np, label, path


# ------------- Backbone ResNet18 -------------
resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet.fc = nn.Identity()
resnet = resnet.to(DEVICE).eval()

for p in resnet.parameters(): 
    p.requires_grad = False

compress = nn.Linear(512, GLOBAL_DIM).to(DEVICE).eval()

img_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

# ------------- YOLO8n silent -------------------
yolo = YOLO("yolov8n.pt")
yolo.to(DEVICE)

def yolo_infer(img_np):
    pil = Image.fromarray(img_np)
    with torch.no_grad():
        return yolo.predict(pil, verbose=False, device=DEVICE)[0]


# -------------- Graph builder ------------------
def build_graph(image_np, yres):
    H, W = image_np.shape[:2]

    with torch.no_grad():
        t = img_transform(Image.fromarray(image_np)).unsqueeze(0).to(DEVICE)
        feat = resnet(t)
    gvec = compress(feat).squeeze(0)

    boxes = getattr(yres, "boxes", None)
    if boxes is None or len(boxes) == 0:
        local = torch.zeros((LOCAL_DIM,), device=DEVICE)
        x = torch.cat([gvec, local]).unsqueeze(0)
        ei = torch.tensor([[0],[0]], device=DEVICE)
        return Data(x=x.float(), edge_index=ei)

    xyxy = boxes.xyxy.cpu().numpy()
    conf = boxes.conf.cpu().numpy()
    cls_ids = boxes.cls.cpu().numpy()

    nodes=[]
    for i in range(len(xyxy)):
        x1,y1,x2,y2 = xyxy[i]
        cx=(x1+x2)/2/W
        cy=(y1+y2)/2/H
        area=((x2-x1)*(y2-y1))/(W*H)

        local = torch.tensor([cls_ids[i], conf[i], cx, cy, area],device=DEVICE)
        nodes.append(torch.cat([gvec, local]))

    x = torch.stack(nodes)
    N = x.size(0)

    if N==1:
        ei = torch.tensor([[0],[0]], device=DEVICE)
    else:
        edges = [[i,j] for i in range(N) for j in range(N) if i!=j]
        ei = torch.tensor(edges,device=DEVICE).t()

    return Data(x=x.float(), edge_index=ei)


# ---------------- Hybrid GAT model (3 output classes) ----------------
class HybridGAT(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(IN_DIM, 256, heads=2)
        self.conv2 = GATConv(512, 256, heads=1)
        self.fc = nn.Linear(256, 3)  # TRAINED ON 3 CLASSES

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = x.mean(dim=0, keepdim=True)
        return self.fc(x)


# ---------------- Load checkpoint ----------------
ckpt = torch.load(CHECKPOINT_PATH, map_location=DEVICE)

model = HybridGAT().to(DEVICE)
model.load_state_dict(ckpt["model_state"])
compress.load_state_dict(ckpt["compress_state"])

model.eval()
compress.eval()

print("[INFO] Model loaded successfully.")


# ---------------- Evaluation ----------------
dataset = EvalDataset(TEST_DIR)
loader = torch.utils.data.DataLoader(dataset, batch_size=1)

y_true=[]
y_pred=[]

for img_np, label, path in tqdm(loader):
    img_np = img_np[0].numpy()
    label = int(label)

    yres = yolo_infer(img_np)
    graph = build_graph(img_np,yres).to(DEVICE)

    with torch.no_grad():
        logits = model(graph.x, graph.edge_index)
        probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]

    # map 3 -> 2 classes
    fire_prob = probs[1]                       # class 1 (fire)
    nonfire_prob = max(probs[0], probs[2])     # normal or accident → non_fire

    pred = 1 if fire_prob > nonfire_prob else 0

    y_true.append(label)
    y_pred.append(pred)


# ---------------- Metrics ----------------
acc = accuracy_score(y_true,y_pred)
prec,rec,f1,_ = precision_recall_fscore_support(y_true,y_pred,average="binary")
cm = confusion_matrix(y_true,y_pred)

print("\n===== RESULTS =====")
print("Accuracy :",acc)
print("Precision:",prec)
print("Recall   :",rec)
print("F1 Score :",f1)
print("\nConfusion Matrix:")
print(cm)


DEVICE = cuda
[INFO] Model loaded successfully.
[EVAL] Loaded 8617 samples.


100%|██████████| 8617/8617 [02:07<00:00, 67.51it/s]



===== RESULTS =====
Accuracy : 0.4068701404200998
Precision: 1.0
Recall   : 0.005061319836480436
F1 Score : 0.010071663761379043

Confusion Matrix:
[[3480    0]
 [5111   26]]


In [ ]:
# Hybrid GNN Training Script (Fire vs Non-Fire)

import os
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from torchvision import models, transforms
from torch_geometric.data import Data
from torch_geometric.nn import GATConv

# Silence ultralytics logs
try:
    from ultralytics.utils import LOGGER as UL_LOGGER
    UL_LOGGER.setLevel(40)
except:
    pass
from ultralytics import YOLO


# CONFIG
DATA_DIR = "data/eval_fire/train"   # 🔥 NEW FOLDER
CLASSES = ["non_fire", "fire"]      # 🔥 ONLY 2 CLASSES NOW
CLASS_MAP = {c: i for i, c in enumerate(CLASSES)}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
YOLO_WEIGHTS = "yolov8n.pt"

EPOCHS = 10
LR = 1e-4
BATCH_SIZE = 1

# 🔥 NEW CHECKPOINT NAME
CHECKPOINT_PATH = "hybrid_fire_checkpoint.pt"

GLOBAL_DIM = 128
LOCAL_DIM = 5
IN_DIM = GLOBAL_DIM + LOCAL_DIM

print("[CONFIG] DEVICE:", DEVICE)


# DATASET
class GNNImageDataset(Dataset):
    def __init__(self, root):
        self.samples = []
        for cname in CLASSES:
            folder = os.path.join(root, cname)
            imgs = glob.glob(folder + "/*")
            for f in imgs:
                if f.lower().endswith((".jpg", ".jpeg", ".png")):
                    self.samples.append((f, CLASS_MAP[cname]))

        print(f"[DATASET] Loaded {len(self.samples)} images.")

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        img_np = np.array(img).astype("uint8")

        # fix dimensionality
        if img_np.ndim == 4:
            img_np = img_np.squeeze()
        if img_np.ndim == 2:
            img_np = np.stack([img_np]*3, axis=-1)
        if img_np.shape[-1] != 3:
            img_np = img_np[..., :3]

        return img_np, label, path

    def __len__(self):
        return len(self.samples)


# RESNET BACKBONE
resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet.fc = nn.Identity()
resnet = resnet.to(DEVICE).eval()
for p in resnet.parameters():
    p.requires_grad = False

compress = nn.Linear(512, GLOBAL_DIM).to(DEVICE)


img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])


# YOLO inference
def yolo_infer(model, img_np):
    pil = Image.fromarray(img_np)
    with torch.no_grad():
        out = model.predict(pil, verbose=False, device=DEVICE)
    return out[0]


# GRAPH BUILDER
def build_graph(image_np, yres):
    """
    Fully safe graph builder.
    Ensures:
      - x has shape (N, IN_DIM)
      - edge_index ALWAYS has shape (2, X) with X>=1
      - never returns empty edge_index
    """
    H, W = image_np.shape[:2]

    # 1) GLOBAL FEATURE (ResNet18 → compress)
    try:
        t = img_transform(Image.fromarray(image_np)).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            feat = resnet(t)
        global_vec = compress(feat).squeeze(0)
    except:
        # catastrophic fallback
        global_vec = torch.zeros(GLOBAL_DIM, device=DEVICE)

    # 2) YOLO DETECTIONS
    boxes = getattr(yres, "boxes", None)

    # ----------- CASE A: NO DETECTIONS ---------------
    if boxes is None or len(boxes) == 0:
        x = torch.cat([global_vec, torch.zeros(LOCAL_DIM, device=DEVICE)], dim=0)
        x = x.unsqueeze(0)  # shape (1, IN_DIM)
        edge_index = torch.tensor([[0],[0]], dtype=torch.long, device=DEVICE)
        return Data(x=x.float(), edge_index=edge_index)

    # Try to extract detection arrays safely
    try:
        xyxy = boxes.xyxy.cpu().numpy()
        conf = boxes.conf.cpu().numpy()
        cls_ids = boxes.cls.cpu().numpy()
    except:
        # fallback to dummy graph
        x = torch.cat([global_vec, torch.zeros(LOCAL_DIM, device=DEVICE)], dim=0)
        x = x.unsqueeze(0)
        edge_index = torch.tensor([[0],[0]], dtype=torch.long, device=DEVICE)
        return Data(x=x.float(), edge_index=edge_index)

    # 3) Build node features
    nodes = []
    for i in range(len(xyxy)):
        x1, y1, x2, y2 = xyxy[i]
        cx = (x1 + x2) / 2 / max(W, 1)
        cy = (y1 + y2) / 2 / max(H, 1)
        area = ((x2 - x1) * (y2 - y1)) / max(W*H, 1)

        local = torch.tensor(
            [float(cls_ids[i]), float(conf[i]), float(cx), float(cy), float(area)],
            device=DEVICE
        )

        nodes.append(torch.cat([global_vec, local], dim=0))

    # If failed to build nodes for some reason
    if len(nodes) == 0:
        x = torch.cat([global_vec, torch.zeros(LOCAL_DIM, device=DEVICE)], dim=0).unsqueeze(0)
        edge_index = torch.tensor([[0],[0]], dtype=torch.long, device=DEVICE)
        return Data(x=x.float(), edge_index=edge_index)

    # stack
    x = torch.stack(nodes).float()
    N = x.size(0)
-
    # 4) Build edges safely
    if N == 1:
        # self-loop only
        edge_index = torch.tensor([[0],[0]], dtype=torch.long, device=DEVICE)
    else:
        # fully connected
        edges = [[i, j] for i in range(N) for j in range(N) if i != j]
        if len(edges) == 0:
            edge_index = torch.tensor([[0],[0]], dtype=torch.long, device=DEVICE)
        else:
            edge_index = torch.tensor(edges, dtype=torch.long, device=DEVICE).t().contiguous()

    return Data(x=x, edge_index=edge_index)



# HYBRID GAT MODEL (2 CLASSES)
class HybridGAT(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(IN_DIM, 256, heads=2)
        self.conv2 = GATConv(512, 256, heads=1)
        self.fc = nn.Linear(256, 2)   # 🔥 ONLY 2 CLASSES NOW

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = x.mean(dim=0, keepdim=True)
        return self.fc(x)


# ============================================================
# TRAINING INITIALIZATION
# ============================================================
yolo = YOLO(YOLO_WEIGHTS)
yolo.to(DEVICE)

dataset = GNNImageDataset(DATA_DIR)
labels = [lbl for _, lbl, _ in dataset]
counts = Counter(labels)
weights = torch.DoubleTensor([1.0/np.sqrt(counts[l]) for l in labels])

sampler = WeightedRandomSampler(weights, len(weights), replacement=True)
loader = DataLoader(dataset, batch_size=1, sampler=sampler, num_workers=2)

model = HybridGAT().to(DEVICE)
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(compress.parameters()),
    lr=LR
)
criterion = nn.CrossEntropyLoss()

best_acc = 0.0


# ============================================================
# TRAINING LOOP
# ============================================================
print("\n===== START TRAINING =====\n")

for epoch in range(1, EPOCHS + 1):
    total_loss = 0
    correct = 0
    total = 0

    pbar = tqdm(loader, desc=f"Epoch {epoch}/{EPOCHS}")
    for img_np, label, _ in pbar:
        img_np = img_np[0].numpy()

        yres = yolo_infer(yolo, img_np)
        graph = build_graph(img_np, yres).to(DEVICE)

        lbl = torch.tensor([label], device=DEVICE)

        optimizer.zero_grad()
        logits = model(graph.x, graph.edge_index)
        loss = criterion(logits, lbl)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = logits.argmax(1).item()
        correct += (pred == label)
        total += 1

        pbar.set_postfix({
            "loss": float(total_loss/total),
            "acc": float(correct/total)
        })

    acc = correct/total
    print(f"[EPOCH {epoch}] Loss={total_loss/total:.4f}  Acc={float(acc):.4f}")

    if float(acc) > best_acc:
        best_acc = float(acc)
        torch.save({
            "model_state": model.state_dict(),
            "compress_state": compress.state_dict(),
            "epoch": epoch,
            "best_acc": best_acc,
            "classes": CLASSES
        }, CHECKPOINT_PATH)
        print(f"[INFO] Saved best model {best_acc:.4f}")

print("\n===== TRAINING COMPLETE =====")
print("Best accuracy:", best_acc)


[CONFIG] DEVICE: cuda
[DATASET] Loaded 39375 images.

===== START TRAINING =====



Epoch 1/10: 100%|██████████| 39375/39375 [10:41<00:00, 61.42it/s, loss=0.032, acc=0.989] 


[EPOCH 1] Loss=0.0320  Acc=0.9886
[INFO] Saved best model 0.9886


Epoch 2/10: 100%|██████████| 39375/39375 [10:28<00:00, 62.64it/s, loss=0.0157, acc=0.995]


[EPOCH 2] Loss=0.0157  Acc=0.9949
[INFO] Saved best model 0.9949


Epoch 3/10:  21%|██▏       | 8457/39375 [02:15<08:18, 61.99it/s, loss=0.0147, acc=0.996] 

In [2]:
#   HYBRID GNN EVALUATION FOR 2-CLASS FIRE MODEL

import os
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models, transforms
from torch_geometric.data import Data
from torch_geometric.nn import GATConv

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from ultralytics import YOLO


# ---------------- CONFIG ----------------
TEST_DIR = "data/eval_fire"     # contains fire / non_fire
CHECKPOINT_PATH = "hybrid_fire_checkpoint.pt"

CLASSES = ["non_fire", "fire"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GLOBAL_DIM = 128
LOCAL_DIM = 5
IN_DIM = GLOBAL_DIM + LOCAL_DIM

print("DEVICE =", DEVICE)


# ---------------- Dataset ----------------
class EvalDataset(torch.utils.data.Dataset):
    def __init__(self, root):
        self.samples = []
        for cname, label in [("non_fire",0), ("fire",1)]:
            folder = os.path.join(root, cname)
            imgs = glob.glob(folder + "/*")
            for f in imgs:
                if f.lower().endswith((".jpg",".png",".jpeg")):
                    self.samples.append((f, label))
        print(f"[EVAL] Loaded {len(self.samples)} samples.")

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        img_np = np.array(img).astype("uint8")
        return img_np, label, path


# ---------------- RESNET BACKBONE ----------------
resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet.fc = nn.Identity()
resnet = resnet.to(DEVICE).eval()
for p in resnet.parameters(): p.requires_grad = False

compress = nn.Linear(512, GLOBAL_DIM).to(DEVICE)


img_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])


# ---------------- YOLO8n ----------------
yolo = YOLO("yolov8n.pt")
yolo.to(DEVICE)

def yolo_infer(img_np):
    pil = Image.fromarray(img_np)
    with torch.no_grad():
        return yolo.predict(pil, verbose=False, device=DEVICE)[0]


# ---------------- Graph Builder ----------------
def build_graph(image_np, yres):
    H, W = image_np.shape[:2]

    # global image feature
    with torch.no_grad():
        t = img_transform(Image.fromarray(image_np)).unsqueeze(0).to(DEVICE)
        feat = resnet(t)
    gvec = compress(feat).squeeze(0)

    boxes = getattr(yres, "boxes", None)

    # No detections → 1 dummy node
    if boxes is None or len(boxes) == 0:
        x = torch.cat([gvec, torch.zeros(LOCAL_DIM, device=DEVICE)], dim=0).unsqueeze(0)
        edge_index = torch.tensor([[0],[0]], dtype=torch.long, device=DEVICE)
        return Data(x=x.float(), edge_index=edge_index)

    xyxy = boxes.xyxy.cpu().numpy()
    conf = boxes.conf.cpu().numpy()
    cls_ids = boxes.cls.cpu().numpy()

    nodes = []
    for i in range(len(xyxy)):
        x1,y1,x2,y2 = xyxy[i]
        cx = (x1+x2)/2/W
        cy = (y1+y2)/2/H
        area = ((x2-x1)*(y2-y1))/(W*H)

        local = torch.tensor([cls_ids[i], conf[i], cx, cy, area], device=DEVICE)
        nodes.append(torch.cat([gvec, local]))

    x = torch.stack(nodes)
    N = x.size(0)

    # fully connected edges
    if N == 1:
        edge_index = torch.tensor([[0],[0]], dtype=torch.long, device=DEVICE)
    else:
        edges = [[i,j] for i in range(N) for j in range(N) if i!=j]
        edge_index = torch.tensor(edges, dtype=torch.long, device=DEVICE).t()

    return Data(x=x.float(), edge_index=edge_index)


# ---------------- Hybrid GAT (2-class) ----------------
class HybridGAT(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(IN_DIM, 256, heads=2)
        self.conv2 = GATConv(512, 256, heads=1)
        self.fc = nn.Linear(256, 2)  # FIRE / NON_FIRE

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = x.mean(dim=0, keepdim=True)
        return self.fc(x)


# ---------------- Load Checkpoint ----------------
ckpt = torch.load(CHECKPOINT_PATH, map_location=DEVICE)

model = HybridGAT().to(DEVICE)
model.load_state_dict(ckpt["model_state"])

compress.load_state_dict(ckpt["compress_state"])

model.eval()
compress.eval()

print("[INFO] Loaded trained fire model.")


# ---------------- Run Evaluation ----------------
dataset = EvalDataset(TEST_DIR)
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

y_true=[]
y_pred=[]

print("\n[EVAL] Running evaluation...\n")

for img_np, label, path in tqdm(loader):
    img_np = img_np[0].numpy()

    yres = yolo_infer(img_np)
    graph = build_graph(img_np, yres).to(DEVICE)

    with torch.no_grad():
        logits = model(graph.x, graph.edge_index)
        pred = logits.argmax().item()

    y_true.append(label)
    y_pred.append(pred)


# ---------------- Metrics ----------------
acc = accuracy_score(y_true, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary")
cm = confusion_matrix(y_true, y_pred)

print("\n===== FINAL RESULTS =====")
print("Accuracy :", acc)
print("Precision:", prec)
print("Recall   :", rec)
print("F1 Score :", f1)
print("\nConfusion Matrix:")
print(cm)


DEVICE = cuda
[INFO] Loaded trained fire model.
[EVAL] Loaded 8617 samples.

[EVAL] Running evaluation...



100%|██████████| 8617/8617 [01:53<00:00, 76.07it/s]



===== FINAL RESULTS =====
Accuracy : 0.6749448764071022
Precision: 0.6649717514124294
Recall   : 0.9164882226980728
F1 Score : 0.7707293116149627

Confusion Matrix:
[[1108 2372]
 [ 429 4708]]
